How to Finetune LLMs with LoRA

Install necessary libraries (PEFT, datasets).
Load a pre-trained language model (bigscience/bloomz-560m) and its tokenizer.
Load the dataset and preprocess it for the model.
Configure LoRA using LoraConfig.
Apply LoRA to the pre-trained model using get_peft_model.
Set up training arguments using TrainingArguments.
Initialize and train the model using Trainer.
Save the fine-tuned LoRA model.
Load the saved LoRA model for inference using PeftModel.from_pretrained.
Generate text using the fine-tuned model and the tokenizer.

Install necessary libraries (PEFT, datasets).
Load a pre-trained language model (bigscience/bloomz-560m) and its tokenizer.

2 Charger le modèle bloomz-560m et son tokenizer \
3️⃣ Charger le dataset Abirate/english_quotes (10% de l'échantillon training)\
4️⃣ Configurer LoRA avec LoraConfig\
5️⃣ Appliquer LoRA sur le modèle avec get_peft_model\
6️⃣ Définir les arguments d’entraînement (TrainingArguments)\
7️⃣ Entraîner le modèle avec Trainer\
8️⃣ Enregistrer et charger le modèle fine-tuné\
9️⃣ Faire de l’inférence (génération de texte)

In [2]:
%pip install peft==0.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
import os
os.makedirs("cache", exist_ok=True)

Chargement du modèle et du tokenizer

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
# Définir le modèle
model_name = "bigscience/bloomz-560m"

In [6]:
# Charger le tokenizer et le modèle pré-entraîné
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)
# AutoModelForCausalLM est utilisé pour les modèles génératifs (comme GPT).

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Chargement et prétraitement du dataset
On utilise Abira

In [7]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.6 MB/s eta 0:00:00


In [8]:
from datasets import load_dataset

In [9]:
# Charger seulement 10% du dataset
data = load_dataset("Abirate/english_quotes", split="train[:10%]")

README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [10]:
# Tokenisation des citations
data = data.map(lambda samples:
                tokenizer(samples["quote"]),
                batched=True
                )

Map:   0%|          | 0/251 [00:00<?, ? examples/s]

In [11]:
# Affichage d'un échantillon pour vérification
train_sample = data.select(range(5))
display(train_sample)

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 5
})

Configuration de LoRA avec LoraConfig

In [12]:
from peft import LoraConfig, get_peft_model

In [13]:
# configurer LoRA avec les bons hyperparamètres.
lora_config = LoraConfig(
    r=8,  # Taille du rang (détermine la complexité des nouvelles matrices entraînées)
    lora_alpha=16,  # Facteur de mise à l'échelle des nouvelles matrices
    target_modules=["query_key_value"],  # Parties du modèle à modifier
    lora_dropout=0.1,  # Dropout pour éviter le surajustement
    bias="none",  # On ne met pas à jour les biais
    task_type="CAUSAL_LM"
)

In [14]:
# Ajout des couches LoRA au modèle de base (the adapter layers)
peft_model = get_peft_model(foundation_model, lora_config)

In [15]:
# Vérification des paramètres entraînables
print(peft_model.print_trainable_parameters())

trainable params: 786,432 || all params: 560,001,024 || trainable%: 0.14043402892063284
None


-->params optimisé parmi ts les params; all params :ts les params retrouvés ds le model.

Définition des arguments d’entraînement

In [16]:
from transformers import TrainingArguments

In [17]:
# définir les hyperparamètres pour l’entraînement du modèle.
# Définir le répertoire de sortie
output_directory = os.path.join("cache", "peft_lab_outputs")

In [18]:
# Définition des arguments d'entraînement (instantié)
training_args = TrainingArguments(
    report_to="none",  # Désactive les logs vers Weights & Biases
    output_dir=output_directory,  # Répertoire de sauvegarde
    auto_find_batch_size=True,  # Trouve automatiquement la bonne taille de batch
    learning_rate=3e-4,  # Apprentissage plus rapide que pour un fine-tuning classique
    num_train_epochs=3,  # Nombre d'époques (passes sur les données)
    use_cpu=False  # On utilise le GPU si disponible
)

Entraînement du modèle avec Trainer

In [19]:
from transformers import Trainer, DataCollatorForLanguageModeling

In [20]:
# Initialisation du Trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=data,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)  # Pas de masquage de mots
)

In [21]:
# Lancement de l'entraînement
trainer.train()

Step,Training Loss


Step,Training Loss


TrainOutput(global_step=189, training_loss=2.9134518255001653, metrics={'train_runtime': 46.507, 'train_samples_per_second': 16.191, 'train_steps_per_second': 4.064, 'total_flos': 129227856470016.0, 'train_loss': 2.9134518255001653, 'epoch': 3.0})

Sauvegarde et chargement du modèle fine-tuné

In [22]:
import time

In [23]:
# Enregistrer le modèle pour pouvoir le réutiliser.
time_now = int(time.time())  # Timestamp pour nommer le fichier
peft_model_path = os.path.join(output_directory, f"peft_model_{time_now}")

trainer.model.save_pretrained(peft_model_path)
print(f"✅ Modèle enregistré à {peft_model_path}")

✅ Modèle enregistré à cache/peft_lab_outputs/peft_model_1742376875


In [26]:
# Charger le modèle fine-tuné pour l'inférence
peft_model = peft_model.from_pretrained(foundation_model, peft_model_path, is_trainable=False)
peft_model.to("cuda")  # Envoyer sur GPU si disponible

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 1024)
        (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear(
                in_features=1024, out_features=3072, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()

Génération de texte avec le modèle fine-tuné

In [27]:
# On teste maintenant la génération de citations.
# Phrase d'entrée
inputs = tokenizer("Two things are infinite: ", return_tensors="pt").to("cuda")

# Génération du texte
outputs = peft_model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=50  # Nombre maximal de tokens générés
)

# Affichage du résultat
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# Le modèle va générer une citation complète à partir de "Two things are infinite:"

Two things are infinite:  time and space. And the other is the infinite number of things that are infinite.”
